In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
from sklearn.externals import joblib
from sklearn import svm
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords

In [29]:
stops = stopwords.words('spanish')
print(stops)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 

In [30]:
def preprocess(X,y):
    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)


   ### text vectorization--go from strings to lists of numbers
    #vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
     #                                stop_words='english')
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                     stop_words=stopwords.words('spanish'))
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)
    #joblib.dump(vectorizer, 'vectorizer.pkl')
    joblib.dump(vectorizer, 'vectorizer2.pkl')

   ### feature selection, because text is super high dimensional and
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    #joblib.dump(selector, 'selector.pkl')
    joblib.dump(selector, 'selector2.pkl')
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [31]:
#with open("training.txt") as f:
with open("espa.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [32]:
def cleanPhrase(phrase):
    return phrase.replace("!","").replace(".","").replace(",","").replace(":","").lower()

In [33]:
features = [cleanPhrase(frase.split("\t")[1]) for frase in content]
targets = [frase.split("\t")[0] for frase in content]

In [34]:
features_train, features_test, labels_train, labels_test=preprocess(features,targets)

In [35]:
lin_svc = svm.LinearSVC(C=100).fit(features_train, labels_train)

In [36]:
print(accuracy_score(labels_test, lin_svc.predict(features_test)))

0.5


In [37]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=30).fit(features_train, labels_train)
print(accuracy_score(labels_test, rfc.predict(features_test)))

0.5


In [38]:
new_phrase="holix"
#vect=joblib.load("vectorizer.pkl")
#select=joblib.load('selector.pkl')
vect=joblib.load("vectorizer2.pkl")
select=joblib.load('selector2.pkl')
features_transformed  = vect.transform([new_phrase])
features = select.transform(features_transformed).toarray()
print(lin_svc.predict(features))

['1']
